<a href="https://colab.research.google.com/github/fuluki/awesome-notebooks-SEO/blob/master/Compartir_GPT_Reviews_Luis_Fern%C3%A1ndez_Gonz%C3%A1lez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Script generado por Luis Fernandez, @HiItsLuis en twitter. Tienes libertad para modificar, mejorar y compartir parte o la totalidad del contenido de este script.
# Eso sí, haz un uso responsable del mismo.

#Instalamos las dependencias necesarias
%%capture
!pip install tiktoken openai

In [ ]:
# Import de librerías
import tiktoken
import requests
from bs4 import BeautifulSoup
import openai

In [ ]:
#RELLENAR: Añade aquí tu API de OpenAI
openai.api_key = "TU-API"
model = "gpt-3.5-turbo"
max_tokens = 2000

# Introduce la URL del producto a analizar
url = 'https://www.thomann.de/es/harley_benton_hb10g_guitar_combo.htm'

## Scraping

In [ ]:
# Si queremos trabajar en bulk tenemos que usar proxies y un generador de headers como https://pypi.org/project/fake-headers/
HEADERS = ({'User-Agent':
			'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
			AppleWebKit/537.36 (KHTML, like Gecko) \
			Chrome/90.0.4430.212 Safari/537.36',
			'Accept-Language': 'es-ES, en;q=0.5'})


def get_html(url):
  # Send a GET request to the product page
  response = requests.get(url, headers=HEADERS)
  soup = BeautifulSoup(response.content, 'lxml')
  return (soup)


def get_product_name(soup):
  product_name = soup.select('h1')[0].text.strip()
  return(product_name)

def get_reviews(soup):
    reviews = []
    review_divs = soup.find_all("div", class_="review-text")
    for review_div in review_divs:
        review_text = review_div.text.strip()
        if (review_text != ""):
          reviews.append(review_text)
    return reviews


In [ ]:
soup = get_html(url)

In [ ]:
product_name = get_product_name(soup)
reviews = get_reviews(soup)

print(product_name)
print(f'Extraidas {len(reviews)} reseñas:')
print("----------------")
for review in reviews:
  print(review)
  print("----------------")

REVIEW: Este ha sido mi primer amplificador de guitarra eléctrica que he tenido, el cual compré junto a una Epiphone Les Paul. Lo he utilizado en casa. Su volumen es más que suficiente para prácticar en casa, nunca lo pondrás a máximo volumen a no ser que quieras iniciar una guerra en tu vecindario. Pros del amplificador: precio principalmente, manejabilidad (es pequeño y ligero) materiales resistentes y en general robusto después de dos años no ha tenido ningún fallo, avería ni si quiera roces estéticos. Los pods son precisos, el canal limpio es muy correcto, la distorsión es correcta para rock, punk, pasada del 8 la ganancia se vuelve horrorosa. Para metal se queda corto el ampli en sí. Es muy agradecido con pedales y pedaleras multiefectos, exprimen el ampli a un nivel que parece impropio de uno tan barato. Como ampli de principante es recomendable ya que no inviertes demasiado y es correcto en general, los pedales alargan su vida útil en lo que exploras diferentes sonidos hasta ele

## Funciones de apoyo al prompt

In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
  """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def limit_reviews_tokens(reviews, max_tokens=1000):
  tokens_sum = 0
  final_reviews = []

  for review in reviews:
    token_number = num_tokens_from_string(review, "cl100k_base")
    if tokens_sum + token_number >= max_tokens:
        break
    final_reviews.append(review)
    tokens_sum += token_number
  print(f'Utilizando {len(final_reviews)} reseñas del total de {len(reviews)} reseñas')
  return final_reviews

def format_reviews_for_prompt(reviews):
  formatted_reviews = ""
  for idx, review in enumerate(reviews):
    formatted_reviews += (f'{idx+1}. {review} \n')
  return formatted_reviews

In [ ]:
final_reviews = limit_reviews_tokens(reviews, max_tokens)

formatted_reviews = format_reviews_for_prompt(final_reviews)


prompt = f'Resume el siguiente listado de Reseñas para "{product_name}" indicando todos los pros y contras basándote en las siguientes reseñas: \n\n {formatted_reviews} \n\n Escribe tu respuesta únicamente en ESPAÑOL.'

messages = [
  {"role": "system", "content": "Eres un experto en analizando reseñas y opiniones de productos"},
  {"role": "user", "content": prompt},
]



# 1000 tokens a 0.0002$
print(f'El mensaje tiene {num_tokens_from_messages(messages, "gpt-3.5-turbo-0301")} tokens y un coste de ${round(num_tokens_from_messages(messages, "gpt-3.5-turbo-0301") / 1000 * 0.0002, 5)}.')

Utilizando 4 reseñas del total de 4 reseñas
El mensaje tiene 1121 tokens y un coste de $0.00022.


In [ ]:
print(prompt)

Resume el siguiente listado de Reseñas para "Harley Benton HB-10G" indicando todos los pros y contras basándote en las siguientes reseñas: 

 1. Este ha sido mi primer amplificador de guitarra eléctrica que he tenido, el cual compré junto a una Epiphone Les Paul. Lo he utilizado en casa. Su volumen es más que suficiente para prácticar en casa, nunca lo pondrás a máximo volumen a no ser que quieras iniciar una guerra en tu vecindario. Pros del amplificador: precio principalmente, manejabilidad (es pequeño y ligero) materiales resistentes y en general robusto después de dos años no ha tenido ningún fallo, avería ni si quiera roces estéticos. Los pods son precisos, el canal limpio es muy correcto, la distorsión es correcta para rock, punk, pasada del 8 la ganancia se vuelve horrorosa. Para metal se queda corto el ampli en sí. Es muy agradecido con pedales y pedaleras multiefectos, exprimen el ampli a un nivel que parece impropio de uno tan barato. Como ampli de principante es recomendable

## Llamada a la API de OpenAI

In [ ]:
response = openai.ChatCompletion.create(
  model=model,
  messages=messages,
  temperature=0.7,
)


In [ ]:
resultado = (response.choices[0].message["content"])
print(resultado)

Pros:
- Precio muy asequible.
- Robusto y resistente.
- Pequeño y ligero, fácil de manejar y transportar.
- Pods precisos y canal limpio correcto.
- Compatible con pedales y pedaleras multiefectos.
- Buen servicio para iniciarse en el mundo de la guitarra eléctrica.

Contras:
- No es adecuado para géneros musicales muy exigentes, como el metal.
- La ganancia se vuelve horrorosa cuando se pasa del 8.
- La tapa posterior no está protegida y puede acumular polvo y otros elementos.
- Algunas unidades pueden generar ruido.
- El sonido no es excepcional, pero es comprensible por su precio.
